# Thermo Examples

# Ideal Rankine Cycle
Determine the efficiency of a Rankine cycle using steam as the working fluid in which the condenser pressure is 10kPa. The boiler pressure is 2MPa. The steam leaves the boiler as saturated vapor.

![ideal rankine cycle](Images/ideal_rankine_cycle.png)

In [142]:
# Given data
p1 = 0.01 # MPa
p2 = 2 # MPa
# state at point 3 is saturated vapor

Look at the pump first, section 1 to 2

In [143]:
# saturated liquid, thus x = 0
sat_liquid=IAPWS97(P=p1, x=0) #saturated liquid with known T
sat_liquid.v

w_p = sat_liquid.v*(p2-p1)*1000
print(f"Work required by pump: {round(float(w_p),1)} kJ/kg")

Work required by pump: 2.0 kJ/kg


In [144]:
h1 = sat_liquid.h
h2 = h1 + w_p
print(f'h2 = {round(h2,1)} kJ/kg')

h2 = 193.8 kJ/kg


Analyze the boiler, section 2 to 3

In [145]:
# saturated vapor, x = 1
sat_vapor=IAPWS97(P=p2, x=1)

p3 = p2
h3 = sat_vapor.h
s3 = sat_vapor.s

q_H = h3-h2

print(f"Heat input by boiler: {round(float(q_H),1)} kJ/kg")

Heat input by boiler: 2604.6 kJ/kg


Analyze the turbine, section 3 to 4

In [146]:
p4 = p1
s4 = s3
sat_vapor4=IAPWS97(P=p4,s=s4)
h4 = sat_vapor4.h

from custom_thermo import custom_thermo as ct
x4 = ct.find_quality(P=p4,s=s4)

w_t = h3-h4
print(f"Work generated by turbine: {round(float(w_t),1)} kJ/kg")
print(f"The quality of the fluid after exiting the turbine is {round(float(x4),4)}")

Work generated by turbine: 791.7 kJ/kg
The quality of the fluid after exiting the turbine is 0.7587


Analysis of the condenser.

In [147]:
q_L = h4 - h1

print(f"Heat rejected by the condenser: {round(float(q_L),1)} kJ/kg")

Heat rejected by the condenser: 1814.8 kJ/kg


Determine the thermal efficiency

In [148]:
eta_th = (w_t-w_p)/q_H*100

print(f"Thermal efficiency is: {round(float(eta_th),1)}%")

Thermal efficiency is: 30.3%


# Rankine with Reheat

Consider a reheat cycle utilizing steam. Steam leaves the boiler and enters the turbine at 4 MPa, 400&deg;C. After expansion in the turbine to 400kPa, the steam is reheated to 400&deg;C and then expanded in the low pressure turbine to 10kPa. Determine the cycle efficiency.  
  
![rankine with reheat](Images/rankine_with_reheat.png)

In [149]:
p3 = 4 # MPa
p2 = p3
T3 = 400 + 273.15 # kelvins
p4 = 0.4 # 400 kPa
p5 = p4
T5 = 400 + 273.15 # kelvins
p6 = 0.01 # 10 kPa
p1 = p6

Start with the pump

In [150]:
# saturated liquid from the condenser
sat_liquid1 = IAPWS97(P=p1,x=0)
v12 = sat_liquid1.v
h1 = sat_liquid1.h

w_p = v12 * (p2 - p1) * 1000
print(f'The specific volume is {round(v12,6)} m^3/kg')
print(f'Pump work is {round(float(w_p),1)} kJ/kg')

The specific volume is 0.00101 m^3/kg
Pump work is 4.0 kJ/kg


In [151]:
h2 = h1 + w_p
print(f'Enthalpy after the pump is {round(float(h2),1)} kJ/kg')

Enthalpy after the pump is 195.8 kJ/kg


Analyzing the high pressure turbine

In [152]:
sat_vap3 = IAPWS97(T=T3,P=p3)
h3 = sat_vap3.h
s3 = sat_vap3.s
s4 = s3
sat_vap4 = IAPWS97(P=p4,s=s4)
h4 = sat_vap4.h

from custom_thermo import custom_thermo as ct
x4 = ct.find_quality(P=p4,s=s4)

w_hpt = h3 - h4
print(f'Quality of intermediate pressure steam: {round(float(x4),4)}')
print(f'The work output by the high pressure turbine is {round(float(w_hpt),1)} kJ/kg')

Quality of intermediate pressure steam: 0.9757
The work output by the high pressure turbine is 528.1 kJ/kg


Analyzing the low pressure turbine

In [153]:
p5 = p4
sat_vap5 = IAPWS97(T=T5, P=p5)
h5 = sat_vap5.h
s5 = sat_vap5.s
s6 = s5
sat_vap6 = IAPWS97(P=p6, s=s6)
h6 = sat_vap6.h
x6 = ct.find_quality(P=p6,s=s6)

w_lpt = h5 - h6
print(f"Quality of low pressure steam: {round(float(x6),4)}")
print(f"The work output by the low pressure turbine is {round(float(w_lpt),1)} kJ/kg")
print(f"Total work output by the turbine is {round(float(w_hpt+w_lpt),1)} kJ/kg")

Quality of low pressure steam: 0.9668
The work output by the low pressure turbine is 769.3 kJ/kg
Total work output by the turbine is 1297.4 kJ/kg


Analyzing the boiler

In [154]:
q_H = (h3 - h2) + (h5 - h4)
print(f'The heat input by the boiler is {round(float(q_H),1)} kJ/kg')

The heat input by the boiler is 3606.1 kJ/kg


Analyzing the condenser

In [155]:
q_L = h6 - h1
print(f'The heat loss by the condenser is {round(float(q_L),1)} kJ/kg')

The heat loss by the condenser is 2312.7 kJ/kg


Calculate the thermal efficiency

In [156]:
net_work_turbine = w_hpt + w_lpt
eta_th = (net_work_turbine - w_p) / q_H * 100
print(f'The thermal efficiency is {round(float(eta_th),1)}%')

The thermal efficiency is 35.9%


# Real Rankine Cycle

A steam power plant operates on a cycle with pressures and temperatures as shown in the picture. The efficiency of the turbine is 86%, and the efficiency of the pump is 80%. Determine the thermal efficiency of this cycle.
![real rankine cycle](Images/real_rankine_cycle.png)

Given variables

In [157]:
from iapws import IAPWS97 #1
import pyromat as pm

p1 = 0.01 # 10 kPa
T1 = 42+273.15
p2 = 5 # MPa
p3 = 4.8 # MPa
T3 = 40+273.15
p4 = 4 # MPa
T4 = 400+273.15
p5 = 3.8 # MPa
T5 = 380+273.15

eta_turbine = 0.86
eta_pump = 0.8

Analyze the pump

In [158]:
# Saturated liquid
sat_liquid1 = IAPWS97(P=p1,x=0)
h1 = sat_liquid1.h
v1 = sat_liquid1.v

w_pump = v1*(p2-p1)/eta_pump * 1000
h2 = h1 + w_pump
print(f"Work of the pump is {round(float(w_pump),1)} kJ/kg")
print(f"Enthalpy after the pump is {round(float(h2),1)} kJ/kg")

Work of the pump is 6.3 kJ/kg
Enthalpy after the pump is 198.1 kJ/kg


Analyze the boiler

In [159]:
# sat_liquid3 = IAPWS97(P=p3,T=T3)
sat_liquid3 = IAPWS97(T=T3,x=0)
h3 = sat_liquid3.h
vapor4 = IAPWS97(P=p4,T=T4)
h4 = vapor4.h

q_H = h4 - h3
print(f"The heat from the boiler is {round(float(q_H),1)} kJ/kg")

The heat from the boiler is 3046.8 kJ/kg


Analyze the turbine

In [160]:
vapor5 = IAPWS97(P=p5,T=T5)
h5 = vapor5.h
s5 = vapor5.s
s6s = s5
p6 = p1

from custom_thermo import custom_thermo as ct
x6 = ct.find_quality(P=p6,s=s6s)

vapor6 = IAPWS97(P=p6,x=x6)
h6s = vapor6.h

w_turbine = eta_turbine*(h5-h6s)
print(f"The quality of the steam coming out of the turbine is {round(float(x6),4)}")
print(f"The work extracted by the turbine is {round(float(w_turbine),1)} kJ/kg")

The quality of the steam coming out of the turbine is 0.8105
The work extracted by the turbine is 894.5 kJ/kg


Analyze the condenser

In [161]:
q_L = h6 - h1
print(f"The heat removed by the condenser is: {round(float(q_L),1)} kJ/kg")

The heat removed by the condenser is: 2312.7 kJ/kg


Calculate the thermal cycle efficiency

In [162]:
eta_th = (w_turbine - w_pump) / q_H * 100
print(f"Thermal Efficiency of the cycle is {round(float(eta_th),1)}%")

Thermal Efficiency of the cycle is 29.2%


# Ideal Rankine Cycle with Feedwater Heater (FWH)

Consider a regenerative cycle using steam as the working fluid. Steam leaves the boiler and enters the turbine at 4 MPa, 400&deg;C. After expansion to 400 kPa, some of the steam is extracted from the turbine to heat the feedwater in an open FWH. The pressure in the FWH is 400 kPa, and the water leaving it is saturated liquid at 400 kPa. The steam not extracted expands to 10 kPa. Determine the cycle efficiency.
![rankine with feedwater heater](Images/ideal_rankine_with_fwh.png)

In [174]:
# 3 saturated liquid
p3 = 0.4 # 400 kPa
p5 = 4 # Mpa
T5 = 400+273.15 # kelvins
p6 = 0.4 # 400 kPa
p7 = 0.01 # 10 kPa

p1 = p7
p2 = p3

Analyze the low pressure pump

In [176]:
from iapws import IAPWS97
sat_liquid1 = IAPWS97(P=p1,x=0)
v12 = sat_liquid1.v
h1 = sat_liquid1.h

w_pump1 = v12*(p2-p1) * 1000
h2 = h1 + w_pump1
print(f"Work into the low pressure pump is: {round(float(w_pump1),1)} kJ/kg")
print(f"The enthalpy at point 2 is: {round(float(h2),1)} kJ/kg")

Work into the low pressure pump is: 0.4 kJ/kg
The enthalpy at point 2 is: 192.2 kJ/kg


Analyze the turbine

In [177]:
sat_vapor5 = IAPWS97(P=p5,T=T5)
h5 = sat_vapor5.h
s5 = sat_vapor5.s
s6 = s5

from custom_thermo import custom_thermo as ct
x6 = ct.find_quality(P=p6,s=s6)

sat_vapor6 = IAPWS97(P=p6,x=x6)
T6 = sat_vapor6.T
h6 = sat_vapor6.h
print(f"The quality of the fluid at point 6 is: {round(x6,4)}")

The quality of the fluid at point 6 is: 0.9757


Analyze the feedwater heater

In [178]:
sat_liquid3 = IAPWS97(P=p3,x=0)
h3 = sat_liquid3.h

The energy conservation equation for the FWH is:
$$y(h_6)+(1-y)h_2=h_3$$
This can be re-arranged to solve y explicitly:
$$y=\frac{h_2-h_3}{h_2-h_6}$$

In [179]:
y = (h2-h3)/(h2-h6)
print(f"The mass fraction of fluid diverted to the FWH from the turbine is\ny = {round(float(y),4)}")

The mass fraction of fluid diverted to the FWH from the turbine is
y = 0.1654


Now we can finish up the turbine analysis

In [180]:
s7 = s5
sat_vapor7 = IAPWS97(P=p7,s=s7)
h7 = sat_vapor7.h
x7 = ct.find_quality(P=p7,s=s7)

w_t = h5 - (h6*y + h7*(1-y))
print(f"The quality of the fluid at point 7 is: {round(float(x7),4)}")
print(f"Work output of the turbine is: {round(float(w_t),1)} kJ/kg")

The quality of the fluid at point 7 is: 0.8163
Work output of the turbine is: 980.3 kJ/kg


Analyze the high pressure pump

In [181]:
v34 = sat_liquid3.v

w_pump2 = v34*(p4-p3) * 1000
print(f"The work done by the high pressure pump is: {round(float(w_pump2),1)} kJ/kg")

The work done by the high pressure pump is: 3.9 kJ/kg


Analyze the boiler

In [182]:
h4 = h3 + w_pump2

q_H = h5 - h4
print(f"Heat input by the boiler is: {round(float(q_H),1)} kJ/kg")

Heat input by the boiler is: 2605.7 kJ/kg


Calculate the thermal efficiency of the cycle

In [184]:
eta_th = (w_t - w_pump1*(1-y) - w_pump2) / q_H * 100
print(f"Thermal efficiency of the cycle: {round(float(eta_th),1)}%")

Thermal efficiency of the cycle: 37.5%
